In [4]:
import cudf
import json

# JSON 파일 경로 설정
file_path = 'merged_data.json'
parquet_file_path = 'merged_data.parquet'

# JSON 파일을 읽어서 리스트로 변환
with open(file_path, 'r', encoding='utf-8') as file:
    json_data = json.load(file)

# jq_schema에 맞춰 데이터를 처리
data = []
for item in json_data:
    processed_item = {
        "filename": item.get("filename"),
        "date": item.get("date"),
        "conference_number": item.get("conference_number"),
        "question_number": item.get("question_number"),
        "meeting_name": item.get("meeting_name"),
        "generation_number": item.get("generation_number"),
        "committee_name": item.get("committee_name"),
        "meeting_number": item.get("meeting_number"),
        "session_number": item.get("session_number"),
        "agenda": item.get("agenda"),
        "law": item.get("law"),
        "qna_type": item.get("qna_type"),
        "context": item.get("context"),
        "context_learn":item.get("context_learn"),
        "context_summary_summary_q": item.get("context_summary", {}).get("summary_q"),
        "context_summary_summary_a": item.get("context_summary", {}).get("summary_a"),
        "questioner_name": item.get("questioner_name"),
        "questioner_affiliation": item.get("questioner_affiliation"),
        "questioner_position": item.get("questioner_position"),
        "question_comment": item.get("question", {}).get("comment"),
        "question_keyword": item.get("question", {}).get("keyword"),
        "answerer_name": item.get("answerer_name"),
        "answerer_affiliation": item.get("answerer_affiliation"),
        "answerer_position": item.get("answerer_position"),
        "answer_comment": item.get("answer", {}).get("comment"),
        "answer_keyword": item.get("answer", {}).get("keyword"),
    }
    data.append(processed_item)

# GPU에서 처리하기 위해 cuDF의 DataFrame으로 변환
df = cudf.DataFrame(data)

# Parquet 파일로 저장
df.to_parquet(parquet_file_path)


In [5]:
df = cudf.read_parquet('merged_data.parquet')

print(df.columns)
df

Index(['filename', 'date', 'conference_number', 'question_number',
       'meeting_name', 'generation_number', 'committee_name', 'meeting_number',
       'session_number', 'agenda', 'law', 'qna_type', 'context',
       'context_learn', 'context_summary_summary_q',
       'context_summary_summary_a', 'questioner_name',
       'questioner_affiliation', 'questioner_position', 'question_comment',
       'question_keyword', 'answerer_name', 'answerer_affiliation',
       'answerer_position', 'answer_comment', 'answer_keyword'],
      dtype='object')


,filename,date,conference_number,question_number,meeting_name,generation_number,committee_name,meeting_number,session_number,agenda,...,questioner_name,questioner_affiliation,questioner_position,question_comment,question_keyword,answerer_name,answerer_affiliation,answerer_position,answer_comment,answer_keyword
0,SRC_16대_2000_2000년10월20일_국정감사_교육위원회_0001(03004...,2000年10月20日(金),030043,0001,국정감사,16,교육위원회,2000,2000년10월20일,감사개시,...,임종석,,위원,"실제로 제가 입수한 자료에 의하면 1997년, 2000년 교육개발원에서 조사한 자료...","비평준화지역, 평준화지역, 향상",조성윤,경기도,교육감,"평준화문제는 지금 성남ㆍ분당지역, 고양ㆍ일산지역, 안양지역, 부천지역 그 외에 이삼...","교육청, 평준화문제, 성남ㆍ분당지역, 고양ㆍ일산지역, 한국교육개발원"
1,SRC_16대_2000_2000년10월23일_국정감사_행정자치위원회-제1반_0001...,2000年10月23日(月),028685,0001,국정감사,16,행정자치위원회-제1반,2000,2000년10월23일,감사개시,...,박종희,,위원,우선 아까 제가 자료요구를 했던 선거구별 단속실적을 좀 다시 보충을 해서 자료로 내...,"선거구별, 단속실적, 형사입건대상, 선거구별 형사입건",배희선,전라남도지방경찰청,청장,"위원님, 즉답 드리겠습니다. 제가 아까 한번 말씀드렸습니다마는 금년에 오고 나서...","전남경찰청, 구조조정, 경찰, 부족"
2,SRC_16대_2000_2000년10월30일_국정감사_법제사법위원회_0001(028...,2000年10月30日(月),028928,0001,국정감사,16,법제사법위원회,2000,2000년10월30일,감사개시,...,조순형,,위원,그러니까 내가 아까 얘기한 부산지검의 명예를 회복하기 위해서 해야 된다는 얘기이고 ...,"부산지검, 검사장, 지역사회, 유착, 다대ㆍ만덕사건, 국정감사, 감사원 감사",송광수,부산지방검찰청,검사장,인사가 된 것이 금년 2월인데 그때는 이영복이가 이미 도주했고 나머지 사람들에 대해...,"승진, 인사, 이영복, 도주, 수사검사"
3,SRC_16대_2000_2000년10월30일_국정감사_산업자원위원회_0001(028...,2000年10月30日(月),028961,0001,국정감사,16,산업자원위원회,2000,2000년10월30일,감사개시,...,김학송,,위원,제 지역구가 경남 진해인데 경남 진해에서 청소년들이 부탄가스 흡입을 하는 과정에서 ...,"경남, 청소년들, 부탄가스, 흡입, 지역구, 진해",김영대,한국가스안전공사,사장,고미제 첨가가 98년도 7월1일 이후로 의무화하도록 되어 있었습니다. 그런데 그 내...,"고미제, 첨가, 혼합, 부탄가스"
4,SRC_16대_2000_2000년10월31일_국정감사_통일외교통상위원회-아주반_00...,2000年10月31日(火),029591,0001,국정감사,16,통일외교통상위원회-아주반,2000,2000년10월31일,감사개시,...,장성민,,위원,그 샘플을 보여 주십시오. 그다음에 노벨상 수상에 대한 것이라든지 남북정상회담에 ...,"대상, 샘플, 노벨상, 수상, 남북정상회담, 대통령의, 대북",장세창,주중화인민공화국대한민국대사관,공사(문화홍보),중국에서 여론조사 형식을 빌어서 조사하는 것에는 애로가 있습니다. 저희들이 96년...,"조사, 중국, 여론조사, 형식, 반응, 언론"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,SRC_21대_제399회_제4차_특별위원회_국회2030부산세계박람회유치지원특별위원회...,2022년8월17일(수),052035,0006,특별위원회,21,국회2030부산세계박람회유치지원특별위원회,제399회,제4차,1. 업무보고가. 산업통상자원부나. 국방부다. 해양수산부라. 기획재정부마. 외교부바...,...,최인호,,위원,그걸 위해서 행정절차 이행기간을 38개월에서 20개월로 줄이겠다 했는데 바로 이 자...,"행정절차, 이행기간, 기재부, 부지, 조성",조승환,해양수산부,장관,부처 간에는 협의가 된 것으로 알고 있습니다.,협의
4396,SRC_21대_제399회_제4차_특별위원회_국회2030부산세계박람회유치지원특별위원회...,2022년8월17일(수),052035,0007,특별위원회,21,국회2030부산세계박람회유치지원특별위원회,제399회,제4차,1. 업무보고가. 산업통상자원부나. 국방부다. 해양수산부라. 기획재정부마. 외교부바...,...,최인호,,위원,예. 적어도 엑스포에 도움이 되는 가덕신공항이 되려면 29년도에 개항이 돼야 된다...,"엑스포, 가덕신공항, 지자체",윤상직,2030부산세계박람회유치위원회,사무총장,제가 답변드리기가 적절한지는 모르겠습니다마는 유치위 차원에서 이러한 여러 부처들에 ...,유치위
4397,SRC_21대_제399회_제4차_특별위원회_국회2030부산세계박람회유치지원특별위원회...,2022년8월17일(수),052035,0008,특별위원회,21,국회2030부산세계박람회유치지원특별위원회,제399회,제4차,1. 업무보고가. 산업통상자원부나. 국방부다. 해양수산부라. 기획재정부마. 외교부바...,...,양금희,,위원,"대단히 수고 많습니다. 이번 9월 달에 BIE에 유치계획서를 제출해야 되는데요,...","부산세계박람회, 개도국, 지원, 민주화, 산업화, 디지털, 선도국",이창양,산업통상자원부,장관,지금 위원님 말씀에 적극적으로 동감합니다. 그래서 이번에 유치계획서 안에 그런 내용...,"유치위원회, 개도국, 지원프로그램, 기재부, 산업화, 메타버스, 엑스포, 익스포저"
4398,SRC_21대_제400회_제5차_특별위원회_국회2030부산세계박람회유치지원특별위원회...,2022년10월26일(수),052258,0033,특별위원회,21,국회2030부산세계박람회유치지원특별위원회,제400회,제5차,1. 2030 부산세계박람회 유치 진행 경과 보고(중간점검)가. 2030 부산세계박...,...,김정호,,위원,"국토부는 아까 말한 대로 기본계획, 전략환경영향평가, 과업지시서 보고 싶습니다. 일...","가덕신공항, 전략환경영향평가, , 월드엑스포부산 , 해상 플로팅 도크",박형준,부산광역시,시장,"추가로 잠깐만 말씀드리면, 사타 이후에 부산시가 2030년 이전에 무조건 개항하여야...",
